In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerActivation
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer


In [2]:
device = torch.device("cpu")

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_path = 'deepset/roberta-base-squad2'

# load model
model = AutoModelForQuestionAnswering.from_pretrained(model_path, output_attentions=True)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.start_logits, output.end_logits, output.attentions

In [5]:
def squad_pos_forward_func(inputs, position=0):
    pred = model(inputs_embeds=inputs)
    pred = pred[position]
    return pred.max(1).values

In [6]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

ref_token_id, sep_token_id, cls_token_id

(1, 2, 0)

In [7]:
def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
        [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)
    
def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = interpretable_embedding.indices_to_embeddings(input_ids)
    ref_input_embeddings = interpretable_embedding.indices_to_embeddings(ref_input_ids)
    
    return input_embeddings, ref_input_embeddings

In [8]:
question, text = "What is important to us?", "It is important to us to include, empower and support humans of all kinds."

In [9]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [10]:
ground_truth = 'to include, empower and support humans of all kinds'

ground_truth_tokens = tokenizer.encode(ground_truth, add_special_tokens=False)
ground_truth_end_ind = indices.index(ground_truth_tokens[-1])
ground_truth_start_ind = ground_truth_end_ind - len(ground_truth_tokens) + 1

In [11]:
input_ids

tensor([[    0,  2264,    16,   505,     7,   201,   116,     2,   243,    16,
           505,     7,   201,     7,   680,     6, 15519,     8,   323,  5868,
             9,    70,  6134,     4,     2]])

In [12]:
start_scores, end_scores, output_attentions = predict(input_ids)
                                #    token_type_ids=token_type_ids, \
                                #    position_ids=position_ids, \
                                #    attention_mask=attention_mask)


print('Question: ', question)
print('Predicted Answer: ', ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

Question:  What is important to us?
Predicted Answer:  Ġto Ġinclude , Ġempower Ġand Ġsupport Ġhumans Ġof Ġall Ġkinds


In [21]:
if torch.__version__ >= '1.7.0':
    norm_fn = torch.linalg.norm
else:
    norm_fn = torch.norm
    
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / norm_fn(attributions)
    return attributions

In [14]:
model.roberta.embeddings.word_embeddings

Embedding(50265, 768, padding_idx=1)

In [15]:
interpretable_embedding = configure_interpretable_embedding_layer(model, 'roberta.embeddings.word_embeddings')

/home/fjiriges/anaconda3/envs/cuBERT/lib/python3.8/site-packages/captum/attr/_models/base.py:188: UserWarning: In order to make embedding layers more interpretable they will be replaced with an interpretable embedding layer which wraps the original embedding layer and takes word embedding vectors as inputs of the forward function. This allows us to generate baselines for word embeddings and compute attributions for each embedding dimension. The original embedding layer must be set back by calling `remove_interpretable_embedding_layer` function after model interpretation is finished. 
  warnings.warn(


In [54]:
layer_attrs_start = []
layer_attrs_end = []

layer_attn_mat_start = []
layer_attn_mat_end = []

input_embeddings, ref_input_embeddings = construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                         token_type_ids=token_type_ids, ref_token_type_ids=ref_token_type_ids, \
                                         position_ids=position_ids, ref_position_ids=ref_position_ids)

for i in range(model.config.num_hidden_layers):
    lc = LayerConductance(squad_pos_forward_func, 
                          model.roberta.encoder.layer[i])
    layer_attributions_start = lc.attribute(inputs=input_embeddings, 
                                            baselines=ref_input_embeddings, 
                                            additional_forward_args=())
    layer_attributions_end = lc.attribute(inputs=input_embeddings, 
                                          baselines=ref_input_embeddings, 
                                          additional_forward_args=())
    
    layer_attrs_start.append(summarize_attributions(layer_attributions_start[0]))
    layer_attrs_end.append(summarize_attributions(layer_attributions_end[0]))

    layer_attn_mat_start.append(layer_attributions_start[1])
    layer_attn_mat_end.append(layer_attributions_end[1])

In [45]:
len(layer_attributions_start), len(layer_attributions_start[0]),len(layer_attributions_start[0][0]), len(layer_attributions_start[0][0][0])

(2, 1, 25, 768)

In [46]:
attributions = layer_attributions_start[0].sum(dim=-1).squeeze(0)

In [49]:
attributions, len(attributions)

(tensor([-1.3100,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.6354,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000], grad_fn=<SqueezeBackward1>),
 25)

In [50]:
attributions = attributions / norm_fn(attributions)

In [51]:
attributions

tensor([-0.6252,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.7805,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000], grad_fn=<DivBackward0>)

In [52]:
layer_attrs_start

[tensor([ 0.0256, -0.5729,  0.0341,  0.0408, -0.1145, -0.0303, -0.3497, -0.0515,
         -0.0216,  0.1048,  0.1301,  0.1270,  0.0524,  0.1302,  0.1871,  0.3055,
          0.2256,  0.3622, -0.0029,  0.2535, -0.0385, -0.0221,  0.2685,  0.1251,
          0.0243], grad_fn=<DivBackward0>),
 tensor([ 0.0318, -0.5343, -0.0524, -0.0074, -0.1024, -0.0116, -0.3164, -0.0766,
          0.0246,  0.0976,  0.1128,  0.1142, -0.0031,  0.2055,  0.3382,  0.2990,
          0.1909,  0.3454, -0.0092,  0.2624, -0.0121, -0.0252,  0.2887, -0.0026,
          0.0574], grad_fn=<DivBackward0>),
 tensor([-0.0284, -0.3818, -0.0947, -0.0760, -0.1365, -0.1028, -0.4160,  0.1624,
         -0.0346,  0.1017,  0.2810,  0.3585,  0.0787,  0.1321,  0.2101,  0.3839,
          0.1219,  0.2913, -0.0268,  0.2041,  0.0560, -0.0177,  0.1124, -0.1031,
          0.0842], grad_fn=<DivBackward0>),
 tensor([ 0.0109, -0.4184, -0.1975, -0.0038, -0.0119, -0.0513, -0.3020,  0.1148,
         -0.0791,  0.1149,  0.4825,  0.3796, -0.0782,  0.1

In [17]:
lc = LayerConductance(squad_pos_forward_func, model.roberta.encoder.layer[0])

In [18]:
layer_attributions = lc.attribute(inputs=input_embeddings, 
                                  baselines=ref_input_embeddings, 
                                  additional_forward_args=())

In [22]:
layer_attrs_start.append(summarize_attributions(layer_attributions[0]))

In [23]:
layer_attrs_start

[tensor([ 0.0256, -0.5729,  0.0341,  0.0408, -0.1145, -0.0303, -0.3497, -0.0515,
         -0.0216,  0.1048,  0.1301,  0.1270,  0.0524,  0.1302,  0.1871,  0.3055,
          0.2256,  0.3622, -0.0029,  0.2535, -0.0385, -0.0221,  0.2685,  0.1251,
          0.0243], grad_fn=<DivBackward0>)]

In [25]:
layer_attn_mat_start.append(layer_attrs_start[1])

IndexError: list index out of range